In [ ]:
!pip install -U sentence-transformers


In [ ]:
import pandas as pd

# Load the uploaded file (adjust filename if needed)
df_questions = pd.read_csv("Questions.csv", usecols=["Id", "Title", "Body", "CreationDate"], nrows=50000)

# Display first few rows
df_questions.head()


In [ ]:
from sentence_transformers import SentenceTransformer

# Load a small, fast, effective sentence embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
# Extract question titles
titles = df_questions['Title'].astype(str).tolist()

# Generate dense vector embeddings
embeddings = model.encode(titles, show_progress_bar=True)


In [ ]:
import numpy as np

np.save("question_titles.npy", titles)
np.save("question_embeddings.npy", embeddings)


In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import uvicorn
import nest_asyncio
from pyngrok import ngrok


In [ ]:
app = FastAPI()

titles = np.load("question_titles.npy", allow_pickle=True)
embeddings = np.load("question_embeddings.npy", allow_pickle=True)

class QueryRequest(BaseModel):
    query: str

@app.post("/search")
def search_questions(req: QueryRequest):
    from sklearn.metrics.pairwise import cosine_similarity
    from sentence_transformers import SentenceTransformer

    model = SentenceTransformer("all-MiniLM-L6-v2")
    query_embedding = model.encode([req.query])

    scores = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = scores.argsort()[::-1][:5]
    top_titles = titles[top_indices]

    return {"top_results": top_titles.tolist()}


In [3]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


In [4]:
from pyngrok import ngrok


In [6]:
!pip install uvicorn


In [7]:
import uvicorn


In [8]:
# Enable nested event loop
nest_asyncio.apply()

# Expose the FastAPI app to the internet using ngrok
public_url = ngrok.connect(9200)
print(f"Your app is live at: {public_url}")

# Start the server
uvicorn.run(app, port=9200)


Your app is live at: NgrokTunnel: "https://a1c83182fdb7.ngrok-free.app" -> "http://localhost:9200"


NameError: name 'app' is not defined

In [ ]:
!ngrok config add-authtoken 30M4sFq07MnnJN7ao11As3FeOoo_64mq5pD6GSnPzk62CY2N


In [ ]:
public_url = ngrok.connect(9200)
print(f"Your app is live at: {public_url}")


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np

app = FastAPI()

titles = np.load("question_titles.npy", allow_pickle=True)
embeddings = np.load("question_embeddings.npy", allow_pickle=True)

from sklearn.metrics.pairwise import cosine_similarity

class Query(BaseModel):
    question: str

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

@app.post("/predict")
def predict(query: Query):
    query_embedding = model.encode([query.question])
    similarity_scores = cosine_similarity(query_embedding, embeddings)[0]
    top_index = similarity_scores.argmax()
    return {
        "most_similar_question": titles[top_index],
        "similarity_score": float(similarity_scores[top_index])
    }


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()
public_url = ngrok.connect(9200)
print("Public URL:", public_url)

uvicorn.run(app, port=9200)


In [ ]:
import requests

url = "https://487ac8fd3afd.ngrok-free.app/search"
data = {"query": "how to use pandas dataframe"}

headers = {
    'Content-Type': 'application/json',
    'ngrok-skip-browser-warning': 'true'
}

try:
    response = requests.post(url, json=data, headers=headers, timeout=10)
    print("Status Code:", response.status_code)

    if response.status_code == 200:
        print("Response JSON:")
        print(response.json())
    else:
        print("Something went wrong:", response.text)

except requests.exceptions.RequestException as e:
    print("Request failed:", e)

